In [1]:
from greensight.utils import DIR_DATA
import pandas as pd
import numpy as np
import json
import pandas as pd

In [2]:
# load DataFrame
s2_path = DIR_DATA / "processed_data/sentinel_two_indices.hdf"

df_data = pd.read_parquet(s2_path)
df_orig = df_data.copy()
df_data

greenbelt  Bath and Bristol                                             \
shape             E06000022                                              
band                   ndwi      ndti          dvi      ndpi      arvi   
2017-04-01        -0.698364 -0.337793  3575.746062  0.337793  0.662791   
2017-05-01        -0.701280 -0.359582  3780.281624  0.359582  0.675456   
2017-06-01        -0.670956 -0.295201  3675.092187  0.295201  0.601951   
2017-07-01        -0.614131 -0.251190  3373.880123  0.251190  0.582856   
2017-08-01              NaN       NaN          NaN       NaN       NaN   
...                     ...       ...          ...       ...       ...   
2024-08-01        -0.602442 -0.211111  2765.168691  0.211111  0.495025   
2024-09-01        -0.605956 -0.213965  2755.027028  0.213965  0.516476   
2024-10-01        -0.615078 -0.259195  2713.886433  0.259195  0.561049   
2024-11-01        -0.599835 -0.240204  2331.800852  0.240204  0.532243   
2024-12-01        -0.597630 -0.239063  2156.893776  0.239063  0.521653   

greenbelt                                                        ...  \
shape                                                            ...   
band               wdvi     pssra      gemi      ndvi      savi  ...   
2017-04-01  3957.022082 -0.337793 -0.479395  0.774909  1.162225  ...   
2017-05-01  4156.170441 -0.359582 -0.664718  0.782279  1.173305  ...   
2017-06-01  4171.652016 -0.295201 -0.933618  0.733432  1.100133  ...   
2017-07-01  3808.096822 -0.251190 -3.865655  0.685344  1.028251  ...   
2017-08-01          NaN       NaN       NaN       NaN       NaN  ...   
...                 ...       ...       ...       ...       ...  ...   
2024-08-01  3377.440772 -0.211111 -0.401834  0.650139  0.975088  ...   
2024-09-01  3309.601806 -0.213965 -0.409297  0.665505  0.998129  ...   
2024-10-01  3144.078124 -0.259195 -0.406479  0.693836  1.040592  ...   
2024-11-01  2722.982283 -0.240204 -0.501182  0.665267  0.997745  ...   
2024-12-01  2543.127812 -0.239063 -0.576855  0.657353  0.985877  ...   

greenbelt       York                                                      \
shape      E06000014                                                       
band           ndi45     tndvi      ipvi       mtci         ri     ireci   
2017-04-01  0.434996  0.419851  0.328669   6.601490   2.472169  0.017848   
2017-05-01  0.699359  0.754130  0.671939  12.688500  30.108144 -0.239381   
2017-06-01       NaN       NaN       NaN        NaN        NaN       NaN   
2017-07-01       NaN       NaN       NaN        NaN        NaN       NaN   
2017-08-01  0.604316  0.621902  0.527854   6.274773   5.812412 -0.067694   
...              ...       ...       ...        ...        ...       ...   
2024-08-01  0.544992  0.549131  0.459687   8.964998   5.281019 -0.049443   
2024-09-01  0.562622  0.564775  0.480541   7.996962   6.476226 -0.066050   
2024-10-01  0.560214  0.588430  0.493619   7.857577   5.811782 -0.084636   
2024-11-01  0.598243  0.618075  0.523736   6.973810   6.902465 -0.062675   
2024-12-01  0.548785  0.590795  0.490100   7.521339   5.363163 -0.094763   

greenbelt                      Burton upon Trent and Swadlincote        
shape                                                  E07000193        
band           s2rep        ci                            msavi2 msavi  
2017-04-01  0.128857  0.419859                               NaN   NaN  
2017-05-01  0.314883  0.754145                               NaN   NaN  
2017-06-01       NaN       NaN                               NaN   NaN  
2017-07-01       NaN       NaN                               NaN   NaN  
2017-08-01  0.226076  0.621914                               NaN   NaN  
...              ...       ...                               ...   ...  
2024-08-01  0.155171  0.549143                               NaN   NaN  
2024-09-01  0.086108  0.564789                               NaN   NaN  
2024-10-01  0.154923  0.588446                               NaN   NaN  
2024-11-01  0

In [3]:
lookup_path = DIR_DATA / "id_lookup/id_lookup.json"
with open(lookup_path, "r") as in_file:
    D_lookup = json.load(in_file)

In [4]:
# create weighting dataframe

# instantiate empty df
df_weights = pd.DataFrame(np.nan, index=df_data.index, columns=df_data.columns )

# apply weighting
for code, dct in D_lookup.items():
    # print(code)
    area = dct["Area_ha"]

    greenbelt_name = D_lookup[code] ["GB_Name"]
    greenbelt_area = float(np.sum([val["Area_ha"] for val in D_lookup.values() if val["GB_Name"] == greenbelt_name]))

    df_weights.loc[:, (slice(None), code, slice(None))] = area/greenbelt_area


df_weights.columns = pd.MultiIndex.from_tuples([(i[0].replace("-", " "), i[1], i[2]) for i in df_weights.columns])
df_weights.columns.names = ["greenbelt", "shape", "band"]
df_weights.head()

greenbelt  Bath and Bristol                                                    \
shape             E06000022                                                     
band                   ndwi      ndti       dvi      ndpi      arvi      wdvi   
2017-04-01         0.345098  0.345098  0.345098  0.345098  0.345098  0.345098   
2017-05-01         0.345098  0.345098  0.345098  0.345098  0.345098  0.345098   
2017-06-01         0.345098  0.345098  0.345098  0.345098  0.345098  0.345098   
2017-07-01         0.345098  0.345098  0.345098  0.345098  0.345098  0.345098   
2017-08-01         0.345098  0.345098  0.345098  0.345098  0.345098  0.345098   

greenbelt                                           ...      York            \
shape                                               ... E06000014             
band           pssra      gemi      ndvi      savi  ...     ndi45     tndvi   
2017-04-01  0.345098  0.345098  0.345098  0.345098  ...  0.801384  0.801384   
2017-05-01  0.345098  0.345098  0.345098  0.345098  ...  0.801384  0.801384   
2017-06-01  0.345098  0.345098  0.345098  0.345098  ...  0.801384  0.801384   
2017-07-01  0.345098  0.345098  0.345098  0.345098  ...  0.801384  0.801384   
2017-08-01  0.345098  0.345098  0.345098  0.345098  ...  0.801384  0.801384   

greenbelt                                                               \
shape                                                                    
band            ipvi      mtci        ri     ireci     s2rep        ci   
2017-04-01  0.801384  0.801384  0.801384  0.801384  0.801384  0.801384   
2017-05-01  0.801384  0.801384  0.801384  0.801384  0.801384  0.801384   
2017-06-01  0.801384  0.801384  0.801384  0.801384  0.801384  0.801384   
2017-07-01  0.801384  0.801384  0.801384  0.801384  0.801384  0.801384   
2017-08-01  0.801384  0.801384  0.801384  0.801384  0.801384  0.801384   

greenbelt  Burton upon Trent and Swadlincote        
shape                              E07000193        
band                                  msavi2 msavi  
2017-04-01                               1.0   1.0  
2017-05-01                               1.0   1.0  
2017-06-01                               1.0   1.0  
2017-07-01                               1.0   1.0  
2017-08-01                               1.0   1.0  

[5 rows x 5400 columns]

In [5]:
# check all greenbelts for each band sum to 1.0
assert np.isclose(df_weights.groupby(level=["greenbelt", "band"], axis=1).sum(), 1.0, atol=1.e-6).all().all()
df_weights.groupby(level=["greenbelt", "band"], axis=1).sum()

/tmp/ipykernel_44572/4025412648.py:2: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  assert np.isclose(df_weights.groupby(level=["greenbelt", "band"], axis=1).sum(), 1.0, atol=1.e-6).all().all()
/tmp/ipykernel_44572/4025412648.py:3: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_weights.groupby(level=["greenbelt", "band"], axis=1).sum()


greenbelt  Bath and Bristol                                                  \
band                   arvi   bi  bi2   ci  dvi gemi gndvi ipvi ireci mcari   
2017-04-01              1.0  1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0   1.0   
2017-05-01              1.0  1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0   1.0   
2017-06-01              1.0  1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0   1.0   
2017-07-01              1.0  1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0   1.0   
2017-08-01              1.0  1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0   1.0   
...                     ...  ...  ...  ...  ...  ...   ...  ...   ...   ...   
2024-08-01              1.0  1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0   1.0   
2024-09-01              1.0  1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0   1.0   
2024-10-01              1.0  1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0   1.0   
2024-11-01              1.0  1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0   1.0   
2024-12-01              1.0  1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0   1.0   

greenbelt   ...  York                                                  
band        ... pssra  pvi reip   ri  rvi s2rep savi tndvi tsavi wdvi  
2017-04-01  ...   1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0   1.0  1.0  
2017-05-01  ...   1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0   1.0  1.0  
2017-06-01  ...   1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0   1.0  1.0  
2017-07-01  ...   1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0   1.0  1.0  
2017-08-01  ...   1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0   1.0  1.0  
...         ...   ...  ...  ...  ...  ...   ...  ...   ...   ...  ...  
2024-08-01  ...   1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0   1.0  1.0  
2024-09-01  ...   1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0   1.0  1.0  
2024-10-01  ...   1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0   1.0  1.0  
2024-11-01  ...   1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0   1.0  1.0  
2024-12-01  ...   1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0   1.0  1.0  

[93 rows x 480 columns]

In [7]:
# save DataFrame
output_path = DIR_DATA / "processed_data/sentinel_two_index_weights.hdf"
df_weights.to_parquet(output_path)